# GoogLeNet Model - Transfer Learning

### Imports

In [ ]:
import torch
import torch.nn.functional as F
import torchvision
from torchvision import transforms as T
from torchvision.models import googlenet, GoogLeNet_Weights
from utils import data, eval, train

### Initialize Dataset

In [ ]:
# Constants
DATA_DIR = 'asl-alphabet/asl_alphabet_train/asl_alphabet_train'
SAVE_MODEL_FILE = 'models/cnn.pt'
SEED = 0
NUM_SAMPLES = 1000
DATASET_SPLIT = [0.7, 0.15, 0.15] # Train / Valid / Test Split
BATCH_SIZE = 100
LEARN_RATE = 5e-4
EPOCHS = 10
IMAGE_SIZE = (100, 100)
WEIGHT_DECAY = 1e-3

In [ ]:
# Seed PyTorch
torch.manual_seed(SEED)

# Image transforms
trans = T.Compose([
    T.ToTensor(),
    T.Resize(IMAGE_SIZE),
    T.Normalize((0.5190, 0.4992, 0.5140), 
                (0.2244, 0.2523, 0.2616))
])

# Initialize datasets
dataset = torchvision.datasets.ImageFolder(
    root=DATA_DIR,
    transform=trans
)

# Split dataset into training, validation and test sets
train_loader, valid_loader, test_loader = data.split_dataset(dataset, NUM_SAMPLES, DATASET_SPLIT, BATCH_SIZE, SEED)

# Check for CUDA GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device} device')

### Model

In [ ]:
# Instantiate model and move to GPU if available
model = torchvision.models.googlenet(weights=GoogLeNet_Weights.IMAGENET1K_V1)
model = model.to(device)

### Training

In [ ]:
# Train model and save model weights and training metrics
metrics = train.train_model(model, train_loader, valid_loader, LEARN_RATE, EPOCHS, device, WEIGHT_DECAY, conv=True)
data.save_model(SAVE_MODEL_FILE, model, metrics)

### Evaluation

In [ ]:
# Load model and show trainig and evaluation metrics
metrics = data.load_model(SAVE_MODEL_FILE, model)
eval.plot_metrics(metrics)

In [ ]:
eval_metrics = eval.eval_model(model, test_loader, device, conv=True)